## Machine Learning Pipeline

In [10]:
import os
os.environ["MODIN_ENGINE"] = "dask"

import modin.pandas as pd
from modin.config import Engine
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score
from xgboost import XGBRegressor

Engine.put("dask")

In [11]:
PATH = "../DATA"
PATHS = [PATH+"/yellow_tripdata_201"+str(i)+"-01.parquet" for i in range(1,4)]

test = "../DATA/yellow_tripdata_2011-01.parquet"
test2 = "../DATA/yellow_tripdata_2011-02.parquet"

df = pd.read_parquet(test2)

In [12]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge             object
airport_fee                      object
dtype: object

In [13]:
print(f"File has {len(df)} entries")
print(df.columns)
print()
print(df.head(20))

File has 14202809 entries
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

    VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0          1  2011-02-01 00:08:34   2011-02-01 00:11:55                1   
1          1  2011-02-01 00:03:54   2011-02-01 00:14:09                1   
2          1  2011-02-01 00:36:39   2011-02-01 00:49:33                1   
3          1  2011-02-01 00:33:20   2011-02-01 00:41:46                4   
4          1  2011-02-01 00:23:43   2011-02-01 00:27:48                1   
5          1  2011-02-01 00:07:02   2011-02-01 00:10:30                1   
6          1  2011-02-01 00:55:42   2011-02-01 01:06:52  

In [14]:
df.insert(loc=0, column='duration', value=(df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds(), allow_duplicates=True)

In [15]:
df.drop(columns=["VendorID","passenger_count","RatecodeID","store_and_fwd_flag","PULocationID","DOLocationID","payment_type","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","airport_fee","tpep_pickup_datetime","tpep_dropoff_datetime"])

Future exception was never retrieved
future: <Future finished exception=PermissionError(13, 'Acesso negado', None, 5, None)>
Traceback (most recent call last):
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\distributed\process.py", line 55, in _call_and_set_future
    res = func(*args, **kwargs)
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\multiprocessing\process.py", line 140, in kill
    self._popen.kill()
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\multiprocessing\popen_spawn_win32.py", line 123, in terminate
    _winapi.TerminateProcess(int(self._handle), TERMINATE)
PermissionError: [WinError 5] Acesso negado
Future exception was never retrieved
future: <Future finished exception=PermissionError(13, 'Acesso negado', None, 5, None)>
Traceback (most recent call last):
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\distributed\process.py", line 55, in _call_and_set_future
    r

Future exception was never retrieved
future: <Future finished exception=PermissionError(13, 'Acesso negado', None, 5, None)>
Traceback (most recent call last):
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\distributed\process.py", line 55, in _call_and_set_future
    res = func(*args, **kwargs)
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\multiprocessing\process.py", line 140, in kill
    self._popen.kill()
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\multiprocessing\popen_spawn_win32.py", line 123, in terminate
    _winapi.TerminateProcess(int(self._handle), TERMINATE)
PermissionError: [WinError 5] Acesso negado


CancelledError: lambda-8c47eb3be0e693c5e25c4fb6be7237e2

CancelledError: lambda-86ce84b04c3c9ce8fae3339908ce3c56

In [16]:
X = df[['duration', 'trip_distance']]
y = df['fare_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

This may cause some slowdown.
Consider scattering data ahead of time and using futures.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.


In [17]:
xgb_model = XGBRegressor()
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py", line 1055, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "c:\Users\vicen\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py", line 521, in _wrap_evaluation_m

In [ ]:
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

##### Hyperparameter tuning

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.3, 0.5],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}

grid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

MemoryError: Unable to allocate 27.4 MiB for an array with shape (3590665,) and data type float64

##### Testing

In [ ]:
y_pred = best_model.predict(X_test)

# Calculating metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("Best Parameters:", best_params)

print(f'Accuracy: {acc}')
print(f'Precision: {prec}')
print(f'Recall: {rec}')
print(f'MAE (Mean Absolute Error): {mae}')
print(f'MSE (Mean Squared Error): {mse}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')